<a href="https://colab.research.google.com/github/con11235/SWCON424_FDA_Project/blob/main/MV-model/MV_model_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mean-Variance Model
본 ipynb는 colab을 사용하여 작성하였습니다.<br>
생성된 주식 파일의 저장 위치는 drive를 마운트하여 '/content/drive/MyDrive/FDA/'의 위치에 저장하였습니다.<br>
파일이 필요한 경우 [공유파일](https://drive.google.com/file/d/1-5WAFWZy7LsHps72jH6HgVMar5MyW7WU/view?usp=sharing)으로 다운 가능합니다.<br>
파일을 생성하는 코드는 [github](https://github.com/con11235/SWCON424_FDA_Project/blob/main/input_data_save.ipynb)에서 확인 가능합니다.


## import module

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

아래는 이후 나오는 값들을 보기 용이하게 하고자 과학적 표기법을 사용하지 않도록 하는 코드입니다.

In [2]:
np.set_printoptions(suppress = True)

## Load raw data

저장된 파일을 불러옵니다.

In [3]:
all_df = pd.read_csv('/content/drive/MyDrive/FDA/all_asset_2010_2020.csv')
all_df['Date'] = all_df.Date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
all_df.set_index('Date',inplace=True)

## User input

사용자로부터 직접 input을 받을 수 있는 코드입니다.<br>
st와 en의 경우 년-월-일 의 형태로 입력받습니다.<br>
unittime과 lookback, rebalance는 년, 월, 주, 일 단위를 선택할 수 있으며, 각 단위에 대한 기간을 숫자로 입력합니다.<br>
예를 들어, 3년(3y), 6개월(6m), 5주(5w), 1일(1d)의 단위로 입력합니다.

In [ ]:
symbols = []
while True:
    name = input("종목코드 : ")
    if name == '0':
        break
    symbols.append(name)
st = input("시작일 0000-00-00: ")
en = input("종료일 0000-00-00: ")
unittime = input("투자 단위기간 : ")
lookback = input("Look back : ")
rebalance = input("Rebalance : ")

코드 실행을 위해 임의로 input을 대입한 값입니다.<br>
symbols는 평가하고자 하는 주식의 코드들을 의미합니다.<br>
st는 투자를 시작할 날짜를 의미합니다.<br>
en는 투자를 종료할 날짜를 의미합니다.<br>
unittime은 backtest에서 확인할 포트폴리오의 단위기간입니다.<br>
lookback은 backtest에서 사용할 lookback의 기간입니다.<br>
rebalance는 backtest에서 rebalancing할 기간의 단위입니다.<br>


In [7]:
symbols = ['AAPL', 'MSFT', 'AMZN']# 'FB', 'GOOGL', 'GOOG', 'TSLA', 'NVDA', 'CMCSA', 'ADBE']
st ='2020-06-01'
en = '2020-10-30'
unittime = '1m'
lookback = '6m'
rebalance = '2m'

이후 날짜 설정을 용이하게 하기 위해 datetime으로 변경하는 부분입니다.

In [8]:
st = datetime(int(st[:4]),int(st[5:7]),int(st[8:10]))
en = datetime(int(en[:4]),int(en[5:7]),int(en[8:10]))
def timestr_to_datetime(s):
  if s[-1] == 'd':
    return timedelta(days=int(s[:-1]))
  elif s[-1] == 'w':
    return timedelta(weeks=int(s[:-1]))
  elif s[-1] == 'm':
    return relativedelta(months=int(s[:-1]))
  elif s[-1] == 'y':
    return relativedelta(years=int(s[:-1]))

unittime = timestr_to_datetime(unittime)
lookback = timestr_to_datetime(lookback)
rebalance = timestr_to_datetime(rebalance)

## Define of Mean-Variance model

분석에 사용할 data를 받아 포트폴리오의 분산을 계산하고, cvxpy를 사용하여 minimize합니다.<br>
이때 들어가는 data는 단위 기간에 맞춰 필요한 값만 가진 data이어야 합니다.<br>
예를 들어 2018년 9월부터 2019년 9월의 데이터를 가지고 2달 단위로 계산한 결과를 얻고싶다면 데이터는 18/9, 18/11, 19/1, 19/3, 19/5, 19/7, 19/9의 데이터만을 가지고있어야 합니다.<br>
input의 cons로 원하는 제약조건을 리스트 형태로 입력받을 수 있습니다.<br>
제약조건의 순서는 상관없지만, 원하는 제약조건의 명칭을 입력해야 제약조건을 사용할 수 있습니다.<br>
제약조건의 종류는 다음과 같습니다.<br>
* 'n-sh' : No Shorting
* 'tr-c' : Transaction Cost

In [9]:
data = all_df[symbols]
data_ret = data.pct_change().dropna()

In [44]:
data_lasso=all_df.iloc[:,:100].dropna(axis=1)
data_ret_lasso = data_lasso.pct_change().dropna()
data_ret_lasso

,AAPL,MSFT,AMZN,GOOGL,GOOG,NVDA,CMCSA,ADBE,NFLX,PEP,INTC,ASML,CSCO,COST,QCOM,TMUS,AVGO,AZN,TXN,AMGN,SNY,SBUX,AMD,INTU,ISRG,MDLZ,BKNG,FISV,GILD,ADP,MELI,AMAT,CSX,MU,EQIX,NTES,LRCX,CME,ATVI,VRTX,ADSK,REGN,ADI,BIDU,LULU,VOD,MNST,ILMN,KDP,AEP,EXC,CTSH,IDXX,ROST,MAR,CTAS,BIIB,KLAC,ERIC,XEL,EA,ALGN,CSGP,MTCH,MCHP,EBAY,VRSK,SNPS,PAYX,WBA,XLNX,ORLY,SBAC,CDNS,SGEN,TROW,PCAR,DXCM
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.001729,0.000323,0.005900,-0.004404,-0.004404,0.014603,-0.013553,0.016446,-0.036836,0.012083,-0.000479,0.007790,-0.004455,-0.003030,0.024073,0.002509,0.007423,-0.020249,-0.005767,-0.008663,-0.012739,0.023427,0.001031,0.000000,-0.004502,0.048852,0.011609,0.004248,-0.000924,-0.005370,0.025499,-0.007693,0.026402,0.029493,-0.009310,0.004561,-0.006770,-0.000818,0.001770,-0.033002,-0.015193,0.019082,-0.001579,-0.010072,0.029764,-0.007331,0.003819,-0.006547,-0.011684,-0.011448,-0.016366,0.011325,0.001664,0.018271,0.005791,-0.001516,-0.004847,-0.011105,0.012618,-0.011859,0.028099,-0.026486,0.004734,0.021941,-0.015764,-0.010460,0.006700,-0.008467,-0.000649,-0.008043,-0.012608,0.001558,0.023884,-0.009804,-0.000949,0.011213,0.018028,0.015187
2010-01-06,-0.015906,-0.006137,-0.018116,-0.025209,-0.025209,0.006396,-0.007169,-0.002122,0.035139,-0.010003,-0.003354,0.009161,-0.006510,0.013000,-0.009778,-0.067585,0.007895,-0.007966,-0.007347,-0.007515,-0.000496,-0.007207,-0.014418,-0.013956,0.014510,0.006951,-0.033016,-0.004230,0.034674,-0.002347,-0.011758,-0.002114,0.003191,0.004476,0.009121,0.001195,-0.004544,-0.006781,-0.005300,-0.017532,0.002373,-0.015139,-0.001897,0.016260,0.021049,-0.007819,0.020796,0.061615,0.022253,0.010133,0.006031,0.000000,0.002400,0.015413,0.002879,0.002656,0.000937,-0.003835,0.017653,0.001920,-0.025723,-0.029428,0.001178,-0.008111,-0.008356,-0.006342,0.012978,-0.001798,0.008117,-0.007567,-0.006783,-0.004409,-0.001944,0.011551,0.027540,-0.015634,0.013148,0.029919
2010-01-07,-0.001849,-0.010400,-0.017013,-0.023280,-0.023280,-0.019597,0.021059,-0.019405,-0.017254,-0.006356,-0.009615,-0.027801,0.004505,-0.004833,0.028992,0.009396,-0.006266,0.010851,0.003116,-0.009157,-0.005462,-0.002562,-0.010449,-0.003292,-0.003752,-0.002416,-0.013465,-0.004653,0.002457,-0.000471,-0.019700,-0.010594,-0.006957,-0.033868,-0.020451,-0.031981,-0.001775,0.001236,-0.023979,-0.012610,0.005525,-0.006068,-0.007921,-0.019079,-0.005846,-0.018827,-0.003975,0.017070,-0.010884,0.008598,0.003308,-0.037820,0.037937,0.041006,0.003947,0.008706,-0.008235,0.005499,0.005102,-0.004312,-0.007701,-0.002860,-0.001882,0.004810,-0.005267,-0.011489,-0.001643,-0.002701,0.015781,0.005991,-0.010044,-0.003126,0.014190,0.019576,-0.005545,-0.000739,0.013241,0.029050
2010-01-08,0.006648,0.006897,0.027077,0.013331,0.013331,0.002161,-0.002946,-0.005422,0.017176,-0.003280,0.011165,-0.023928,0.005299,-0.007201,0.010004,-0.055851,0.007357,0.004294,0.022912,0.008886,0.003245,-0.003425,-0.004224,0.026420,-0.005504,0.001038,0.000370,-0.011179,-0.007354,-0.001413,0.004974,0.038544,0.048439,0.023985,-0.004847,-0.018738,0.025152,0.026718,-0.008189,-0.020000,0.030612,-0.008140,0.005749,-0.000890,-0.001238,-0.015171,0.004989,0.011596,-0.007222,0.011935,-0.004739,0.028986,-0.007984,-0.019369,0.002144,0.009756,0.020570,0.015039,-0.004061,0.000481,0.019956,0.013196,-0.001886,0.006223,0.006001,0.012054,-0.012833,0.007223,0.000634,0.001354,0.014611,-0.008884,0.001097,0.014400,0.005576,-0.003881,0.001307,-0.018458
2010-01-11,-0.008822,-0.012720,-0.024041,-0.001512,-0.001512,-0.014016,-0.006501,-0.013083,-0.001313,-0.001152,0.005761,-0.021226,-0.002838,0.001181,-0.003639,0.012676,0.006260,0.017956,-0.012908,0.004403,0.016671,-0.003007,-0.030753,0.004505,0.007541,-0.004493,0.012488,0.005344,0.007858,0.004480,-0.096219,0.021993,-0.009927,-0.018018,0.008242,-0.019849,0.026766,-0.000200,-0.000917,0.028522,-0.000381,0.019286,-0.005716,-0.009152,-0.014255,0.014499,0.000000,0.020211,-0.006581,0.010390,0.011594,-0.008110,0.033268,0.004882,

In [61]:
def mv_opt(data,ret_target = None,cons=[''],lamd=0):
  n = len(data.T)   # asset number
  W = cp.Variable(n)  # weight of assets
  R = data.mean()*252
  C = data.cov()*252
  constraints = [cp.sum(W) == 1]
  
  if ret_target != None:
    ret_gvm = np.dot(R.T, mv_opt(data)/100)
    ret_target = np.round(ret_gvm,2) + ret_target
  else:
    ret_target = 0
  constraints.append(W @ R >= ret_target)
  if 'n-sh' in cons:
    constraints.append(W >= 0)
    print('no shorting')
  if 'lasso' in cons:
    objective = cp.Minimize(cp.quad_form(W, C)+lamd*cp.norm(W,1))
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS)
    return W.value.round(6)*100
  if 'elastic' in cons:
    if (type(lamd) == int) or (type(lamd) == float):
      lamd = (lamd,lamd)
    objective = cp.Minimize(cp.quad_form(W, C)+lamd[0]*cp.norm(W,1)+lamd[1]*(cp.norm(W,2)**2))
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS)
  objective = cp.Minimize(cp.quad_form(W, C))
  problem = cp.Problem(objective, constraints)
  problem.solve(solver=cp.ECOS)
  return W.value.round(6)*100

In [80]:
mv_opt(data_ret_lasso,0.05,cons=['elastic','n-sh'],lamd=0.001)

no shorting


array([ 1.2856,  0.    ,  0.167 ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  1.8213,  6.0803,  0.    ,  0.    ,  0.    , 20.0939,
        0.    ,  0.    ,  0.    ,  8.2784,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        2.777 ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        3.3932,  0.    ,  0.    ,  1.3916,  0.    ,  0.    ,  0.0757,
        0.    ,  0.    ,  0.    ,  0.    ,  0.0866,  0.    , 14.5568,
       16.3603,  0.    ,  0.    ,  1.2109,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  6.8498,  0.    ,  0.    ,  0.    ,
        2.5587,  0.    ,  0.9079,  6.3953,  0.    ,  0.    ,  0.    ,
        0.    ,  4.2484,  1.4614,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ])

In [62]:
mv_opt(data_ret_lasso,cons=['elastic','n-sh'],lamd=0.001)

no shorting


array([ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.0342, 13.8337,  0.    ,  0.    ,  0.    , 18.0937,
        0.    ,  0.    ,  0.    ,  9.6365,  0.    ,  0.    ,  0.0897,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        3.6865,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        2.7324,  0.    ,  0.    ,  1.6329,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.6225,  0.    ,  0.    , 12.898 ,
       18.638 ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  4.1145,  0.    ,  0.    ,  0.    ,
        0.4982,  0.    ,  2.2929,  7.3765,  0.    ,  0.    ,  1.3214,
        0.    ,  2.4982,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ])

In [55]:
mv_opt(data_ret_lasso,cons=['lasso','n-sh'],lamd=0.001)

no shorting


array([ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.0342, 13.8337,  0.    ,  0.    ,  0.    , 18.0937,
        0.    ,  0.    ,  0.    ,  9.6365,  0.    ,  0.    ,  0.0898,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        3.6865,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        2.7324,  0.    ,  0.    ,  1.6329,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.6225,  0.    ,  0.    , 12.898 ,
       18.638 ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  4.1145,  0.    ,  0.    ,  0.    ,
        0.4982,  0.    ,  2.2929,  7.3765,  0.    ,  0.    ,  1.3214,
        0.    ,  2.4982,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ])

In [56]:
mv_opt(data_ret_lasso,cons=['lasso'],lamd=0.001)

array([  2.7025,  -2.159 ,   0.3542,   0.    ,   2.7122,  -2.5083,
         1.0584,  -2.4892,   0.6308,  13.3502,   0.    ,   0.    ,
         0.4764,  16.5664,   0.8882,  -0.0393,  -1.3121,   9.3855,
         0.    ,   0.    ,   2.3223,  -0.    ,  -1.2451,  -4.5605,
         0.    ,   0.    ,   0.6327,  -1.3225,   4.3641,  -0.2551,
        -2.5303,  -0.3938,  -0.0213,  -2.3183,  -0.1234,   3.6959,
        -3.0657,   3.1501,   3.1719,  -0.3953,  -3.4574,  -0.54  ,
         0.    ,   1.7354,  -0.    ,   2.3297,   0.    ,  -0.    ,
        11.0984,  17.9327,  -0.    ,  -2.5454,   2.0557,   0.3477,
         3.4118,   0.    ,   0.395 ,  -2.5269,  -0.    ,   2.7766,
        -0.    ,  -1.8275,   0.2511,   2.2778,  -0.    ,   4.3619,
         9.078 ,   9.0472,  -0.    ,   2.7669,   5.7528,   3.5   ,
         1.7294,  -0.9945,  -0.    , -12.0621,   2.6877,  -0.3046])

In [52]:
mv_opt(data_ret_lasso,cons=['lasso'],lamd=0.005)

array([ 0.2788, -0.    ,  0.    ,  0.    ,  0.3453, -1.2283,  0.    ,
       -0.    ,  0.3504, 13.5624,  0.    , -0.    ,  0.    , 17.8654,
        0.    ,  0.    , -0.    ,  9.5613,  0.    ,  0.    ,  0.8574,
        0.    , -0.731 , -0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        3.9017,  0.    , -1.012 , -0.    ,  0.    , -1.966 ,  0.    ,
        3.2883, -1.0408,  0.    ,  2.2229,  0.    , -0.8596,  0.    ,
        0.    ,  0.5198,  0.    ,  1.3332,  0.    ,  0.    , 12.6401,
       18.3134,  0.    , -0.    ,  0.    ,  0.1468,  0.3991,  0.    ,
        0.    , -0.    ,  0.    ,  3.1938,  0.    , -0.1333,  0.    ,
        1.2507, -0.    ,  3.2008,  7.9567,  1.3949,  0.    ,  1.7965,
        1.1851,  2.761 ,  0.6769, -0.    ,  0.    , -2.0316,  0.    ,
        0.    ])

In [32]:
mv_opt(data_ret,cons=['tr-c'],trans_cost =[0.02,0.01,0.01])

array([33.3333, 36.0272, 30.6395])

In [14]:
mv_opt(data_ret)

array([31.5275, 49.713 , 18.7594])

In [ ]:
mv_opt(data_ret,0,cons=['n-sh'])

array([12.3134,  8.6558,  7.7894,  3.7616, 19.5772,  0.1847,  0.    ,
        0.    , 46.1928,  1.5252])

In [ ]:
mv_opt(data_ret,0.05,cons=['n-sh'])

array([11.1032,  8.6595, 12.8065,  3.7932,  7.0797,  4.7863,  4.197 ,
        1.7578, 38.4879,  7.3288])

## Define of Backtest

backtest를 진행하는 함수입니다.<br>
원하는 기간과 단위 기간에 맞는 데이터를 가져온 후, pct_change 값을 계산합니다.<br>
결과는 각 종목별 amount, 전체 amount, 포트폴리오의 return, 각 종목별 weight 순으로 반환합니다.<br>
dropna는 기본값으로 True로 되어있는데 이 경우 모든 기간동안 weight가 0이었던 종목은 제외하고 결과를 보여줍니다.<br>
이후 rebalancing 기간마다 weight를 계산하고, 단위 기간마다 투자 결과를 보여주는 표를 결과로 반환합니다.<br>

In [ ]:
def backtest(symbols, st, en, ut,lb, rb,dropna=True):
  def data_by_dates():
    data = all_df[symbols]
    df = pd.DataFrame(columns = symbols)
    date = st-lookback
    while date < en:
        df.loc[data[date:].index[0]] = data[date:].values[0]
        date += ut
    rb_date = []
    date = st
    while date < en:
        rb_date.append(data[date:].index[0])
        date += rb
    return df, rb_date
  df, rb_date = data_by_dates()
  df_ret = df.pct_change().dropna()
  
  col = [i+'_amt' for i in symbols]
  col.append('total_amt')
  col.append('portfolio_ret')
  col+= [i+'_weight' for i in symbols]
  result = pd.DataFrame(columns = col)
  T_amt = 100

  for index, row in df.iterrows():
    if st <= index:
      if index in rb_date:
        weight = mv_opt(df_ret[index-lb:index])
      amt = (df_ret[index:index]+1)*weight*T_amt/100
      for i in range(len(symbols)):
        s = symbols[i]
        result.loc[index,s+'_amt'] = amt[s].values[0]
        result.loc[index,s+'_weight'] = weight[i]
      T_amt_ = T_amt
      T_amt = amt.sum(axis = 1).values[0]
      result.loc[index,'total_amt'] = T_amt
      result.loc[index,'portfolio_ret'] = T_amt/T_amt_ - 1
  if dropna == True:
    return result.loc[:,result.eq(0).sum(axis=0) != len(result)]
  return result


In [ ]:
backtest(symbols, st, en, unittime, lookback, rebalance)

,MSFT_amt,AMZN_amt,NVDA_amt,CMCSA_amt,total_amt,portfolio_ret,MSFT_weight,AMZN_weight,NVDA_weight,CMCSA_weight
2020-06-01,0.69662,96.1314,12.9579,0,109.786,0.0978595,0.6633,88.9343,10.4024,0
2020-07-01,0.815318,113.745,12.3646,0,126.925,0.156114,0.6633,88.9343,10.4024,0
2020-08-03,6.43929,102.153,30.4307,0,139.023,0.0953172,4.7959,74.4521,20.752,0
2020-09-01,7.01471,116.385,36.2259,0,159.626,0.148198,4.7959,74.4521,20.752,0
2020-10-01,0,103.084,0,49.121,152.205,-0.0464933,0,70.1485,0,29.8515
